## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意（ここでは説明のため `buyer_id` 昇順で返す）
* `NOT IN` は不使用
* 判定は ID 基準（`buyer_id`・`user_id`）、表示は仕様どおり

---

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  各ユーザについて、**参加日 (`join_date`) と、2019 年に buyer として行った注文数** を求める。

* `{{TABLES_OR_SCHEMAS}}`

  **Users**

  | Column Name    | Type    | Note        |
  | -------------- | ------- | ----------- |
  | user_id        | int     | PK          |
  | join_date      | date    | ユーザの参加日     |
  | favorite_brand | varchar | お気に入りブランド情報 |

  **Orders**

  | Column Name | Type | Note                      |
  | ----------- | ---- | ------------------------- |
  | order_id    | int  | PK                        |
  | order_date  | date | 注文日                       |
  | item_id     | int  | `Items.item_id` への FK     |
  | buyer_id    | int  | 購入者。`Users.user_id` への FK |
  | seller_id   | int  | 販売者。`Users.user_id` への FK |

  **Items**

  | Column Name | Type    | Note  |
  | ----------- | ------- | ----- |
  | item_id     | int     | PK    |
  | item_brand  | varchar | ブランド名 |

  ※本問では `Items` は参照不要。

* `{{OUTPUT_COLUMNS_AND_RULES}}`

  出力カラム:

  | Column Name    | 説明                      |
  | -------------- | ----------------------- |
  | buyer_id       | `Users.user_id` と同一     |
  | join_date      | Users の参加日              |
  | orders_in_2019 | 2019 年に buyer として行った注文数 |

  ルール:

  * **すべてのユーザ** を 1 行ずつ出力する（2019 年の注文が 0 件でも行を出す）。
  * 2019 年に注文がないユーザは `orders_in_2019 = 0` とする。
  * 並び順は任意（ここでは `buyer_id` 昇順）。

---

## 2) 最適解（単一クエリ）

> 2019 年の注文を結合条件側で絞り込んだ `LEFT JOIN` ＋ `GROUP BY` でシンプルに実装します。

```sql
SELECT
  u.user_id AS buyer_id,
  u.join_date,
  COUNT(o.order_id) AS orders_in_2019
FROM Users AS u
LEFT JOIN Orders AS o
  ON  o.buyer_id = u.user_id
  AND o.order_date >= DATE '2019-01-01'
  AND o.order_date <  DATE '2020-01-01'
GROUP BY
  u.user_id,
  u.join_date
ORDER BY
  buyer_id;

Runtime 515 ms
Beats 76.62%

```

---

## 3) 要点解説

1. **LEFT JOIN で「全ユーザ＋注文 0 件ユーザ」を両立**

   * 主テーブル: `Users`（全ユーザを軸にする）
   * `Orders` に対して `LEFT JOIN` することで、2019 年に注文がないユーザでも行が残る。
   * `INNER JOIN` にすると 1 件も注文がないユーザが落ちてしまうので **NG**。

2. **日付フィルタは `JOIN ... ON` 句に書く**

   * 2019 年の範囲を次のように絞っている:

     ```sql
     o.order_date >= DATE '2019-01-01'
     AND o.order_date <  DATE '2020-01-01'
     ```

   * これを `WHERE` 句に書いてしまうと、`LEFT JOIN` の「NULL 行」も除外されてしまい、注文 0 件ユーザが消える。

   * そのため、**日付条件は `ON` 句側に置く**のが正解。

3. **COUNT で 2019 年注文数を集計**

   * `COUNT(o.order_id)` は

     * 2019 年に該当する行だけ数える（`ON` 句でフィルタ済み）
     * 1 件も注文がないユーザは `o.order_id` がすべて `NULL` なので `COUNT` は 0 になる。
   * `GROUP BY u.user_id, u.join_date` でユーザ単位に集計している。

4. **Items テーブルは不要**

   * 問題文の要件（2019 年 buyer としての注文数）だけなら、ブランド情報は使わないため `Items` は結合しない。

---

## 4) 計算量（概算）

* `Users` の行数を `U`, `Orders` の行数を `O` とすると、

  * ハッシュ結合ベースの `LEFT JOIN` ＋ `GROUP BY` は概ね **O(U + O)**。
* `Orders(buyer_id, order_date)` などにインデックスがあると、プランにより 2019 年部分だけを効率的にスキャンできるため、実効コストはさらに下がる。

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Users<br/>全ユーザ] --> B[Orders<br/>2019年だけをON句で結合]
  B --> C["GROUP BY user_id, join_date<br/>COUNT(order_id)"]
  C --> D["出力<br/>buyer_id, join_date,<br/> orders_in_2019"]
```

このクエリをそのまま PostgreSQL 16.6+ に流せば、サンプルの入出力例と同じ結果が得られます。

ここからは「改善の余地があるとしたら」の観点で、いくつかバリエーションとチューニング案を挙げます。

---

## 1) 集約 `FILTER` を使った書き方（ロジックの見通し改善）

もしクエリが「2018 年の件数もほしい」「ブランド別の件数もほしい」など、カウント条件が増えていく可能性があるなら、
**JOIN 条件はシンプルにして、`COUNT(…) FILTER (WHERE …)` に条件を寄せる**のもアリです。

```sql
SELECT
  u.user_id AS buyer_id,
  u.join_date,
  COUNT(o.order_id) FILTER (
    WHERE o.order_date >= DATE '2019-01-01'
      AND o.order_date <  DATE '2020-01-01'
  ) AS orders_in_2019
FROM Users AS u
LEFT JOIN Orders AS o
  ON o.buyer_id = u.user_id
GROUP BY
  u.user_id,
  u.join_date
ORDER BY
  buyer_id;

Runtime 518 ms
Beats 73.06%

```

### この書き方のメリット

* `JOIN` 条件は「関連づけの条件（buyer_id が一致しているか）」だけに集中
* 「いつの期間の注文をカウントするか」は `FILTER (WHERE …)` で一目瞭然
* 例えば、将来こうしたいときに自然に書けます:

  ```sql
  COUNT(o.order_id) FILTER (WHERE order_date >= '2018-01-01' AND order_date < '2019-01-01') AS orders_in_2018,
  COUNT(o.order_id) FILTER (WHERE order_date >= '2019-01-01' AND order_date < '2020-01-01') AS orders_in_2019
  ```

ロジックの分離がはっきりして、「関連づけ」と「ビジネスルール」を分けたい場面ではこちらの方が好まれることも多いです。

---

## 2) インデックス設計（パフォーマンス改善）

データ量が増えてきた場合は、`Orders` 側にインデックスを張っておくとかなり効きます。

### おすすめ候補

```sql
CREATE INDEX idx_orders_buyer_date
  ON Orders (buyer_id, order_date);
```

理由:

* このクエリでは `buyer_id` で結合し、その中で `order_date` 範囲を絞り込みたいので、複合インデックス `(buyer_id, order_date)` がマッチします。
* PostgreSQL はこのインデックスを使って「特定の buyer ごとの 2019 年分だけ」を効率よく読み取れます。

もし「年単位で集計するクエリ」が多いのであれば、`order_date` をキーにパーティショニングするのも検討対象になります。

---

## 3) ORDER BY が不要なら削る

要件として **「並び順: 任意」** であり、アプリケーション側でも特に `ORDER BY` を使っていない場合は、

```sql
ORDER BY buyer_id;
```

は削ってしまっても構いません。
ソートはそこそこコストがかかるので、不要であれば削った方がクエリ自体は軽くなります。

---

## 4) 可読性の微調整（チーム開発向け）

チームで読むことを考えると、次のような小さな工夫も有効です。

### 例: 日付の開始・終了を名前付きにする

```sql
WITH params AS (
  SELECT
    DATE '2019-01-01' AS start_2019,
    DATE '2020-01-01' AS end_2019
)
SELECT
  u.user_id AS buyer_id,
  u.join_date,
  COUNT(o.order_id) FILTER (
    WHERE o.order_date >= p.start_2019
      AND o.order_date <  p.end_2019
  ) AS orders_in_2019
FROM Users AS u
CROSS JOIN params AS p
LEFT JOIN Orders AS o
  ON o.buyer_id = u.user_id
GROUP BY
  u.user_id,
  u.join_date;
```

* 「2019 年という期間」を `params` CTE にまとめることで、
  年度が変わったときの修正漏れを防ぎやすくなります。

---

## 5) 結論

* **ロジック的には現状のクエリで十分正しく・シンプル** です ✅
* よりよくするなら:

  1. `COUNT(...) FILTER (WHERE ...)` でカウント条件を分離 → 今後の拡張性アップ
  2. `Orders(buyer_id, order_date)` の複合インデックス → 大量データ時のパフォーマンス改善
  3. `ORDER BY` が不要なら削除 → ソートコスト削減
  4. 期間を CTE などでまとめておく → 読みやすさ・保守性向上


